In [1]:
import re
import math
import warnings
import tqdm
import copy
import matplotlib
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
from scipy.optimize import fsolve
from sympy import symbols, Eq, solve
from sklearn.preprocessing import MinMaxScaler
from npp import newff, train, sim
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn as nn
import torch.optim as optim
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def select(_a, fit_g, fit_g_n):
    # 计算每个个体的适应度分数（假设适应度越高分数越大）
    fitness1 = 10.0 / fit_g
    # 将适应度分数归一化为概率
    probabilities = fitness1 / np.sum(fitness1)
    # 根据概率随机选择个体的索引，以创建新的群体
    selected_indices = np.random.choice(np.arange(len(probabilities)), size=fit_g_n, p=probabilities)
    # 使用选定的索引从原始群体中获取新的个体和对应的适应度
    new_chrom = _a[selected_indices]
    new_fitness = fit_g[selected_indices]
    # 返回新的群体和相应的适应度
    return new_chrom, new_fitness

In [3]:
def mutation(mutation_prob, lenchrom, _a, fit_g_n, limit):
    # 创建一个空列表，用于存储经过突变后的新个体
    set_new = []

    # 遍历适应度较高的个体
    for i in range(fit_g_n):
        # 根据突变概率决定是否对该个体进行突变
        if np.random.rand() <= mutation_prob:
            # 随机选择一个个体进行突变
            index = np.random.randint(fit_g_n)
            # 复制被选中个体的基因信息
            temp_new = _a[index].copy()
            
            # 计算突变次数，通常为染色体总长度的十分之一
            mu_times = int(np.sum(lenchrom) / 10)
            
            # 随机选择染色体上的位置，并将该位置的基因值修改为在限定范围内的随机值
            for _ in range(mu_times):
                pos = np.random.randint(np.sum(lenchrom))
                temp_new[1, pos] = int(limit[pos, 1]) + np.random.rand()
            
            # 将突变后的个体加入新个体列表
            set_new.append(temp_new)
    
    # 将原有个体和突变后的个体合并成一个新种群
    new_chrom = np.vstack((_a, set_new))
    return new_chrom


In [4]:
# 定义初始化函数，用于创建神经网络初始参数种群
# hidden_num: 隐藏层神经元数目
# x_train_regular: 输入数据
# y_train_regular: 输出数据
# NIND: 种群数量

def initialize(hidden_num, x_train_regular, y_train_regular, NIND):
    pre = []  # 用于存储初始化的神经网络参数种群

    # 循环创建NIND个个体
    for iter in range(NIND):
        pre_chrom = []  # 用于存储单个个体的参数
        # 创建新的前馈神经网络
        pre_net = newff(x_train_regular, y_train_regular, hidden_num, {'tansig', 'purelin'}, 'trainlm')
        
        # 设置训练参数
        pre_net.trainParam.epochs = 200  # 迭代次数
        pre_net.trainParam.lr = 0.1  # 学习率
        pre_net.trainParam.goal = 0.00001  # 训练目标误差
        
        # 提取输入层到隐藏层的权重和偏差
        pre_w1 = pre_net.iw[1, 1]
        pre_chrom = np.append(pre_chrom, pre_w1.flatten())
        pre_chrom = np.append(pre_chrom, pre_net.b[1])
        
        # 提取隐藏层到输出层的权重和偏差
        pre_w2 = pre_net.lw[2, 1]
        pre_chrom = np.append(pre_chrom, pre_w2.flatten())
        pre_chrom = np.append(pre_chrom, pre_net.b[2])
        
        # 将单个个体的参数添加到种群中
        pre.append(pre_chrom)

    # 复制种群，用于计算参数的范围
    ran = copy.deepcopy(pre)
    ran_min = np.min(pre, axis=0)
    ran_max = np.max(pre, axis=0)
    
    # 将范围信息存储在ran中
    ran[0] = ran_min
    ran[1] = ran_max
    
    # 返回初始化的神经网络参数种群
    return pre


In [7]:
import numpy as np  # 导入NumPy库，用于矩阵和数组操作

def fitness(_a, input_num, hidden_num, output_num, input_data, output_data):
    # 从_a中提取权重矩阵W1、偏置B1、权重矩阵W2和偏置B2
    w1 = _a[0:input_num * hidden_num]  # 提取输入层到隐藏层的权重矩阵W1
    B1 = _a[input_num * hidden_num:input_num * hidden_num + hidden_num]  # 提取隐藏层的偏置B1
    w2 = _a[input_num * hidden_num + hidden_num:input_num * hidden_num + hidden_num + hidden_num * output_num]  # 提取隐藏层到输出层的权重矩阵W2
    B2 = _a[input_num * hidden_num + hidden_num + hidden_num * output_num:input_num * hidden_num + hidden_num + hidden_num * output_num + output_num]  # 提取输出层的偏置B2

    # 将权重矩阵W1和W2重新形状成相应的矩阵
    W1 = w1.reshape((hidden_num, input_num))
    W2 = w2.reshape((output_num, hidden_num))

    # 将偏置B1和B2重新形状成相应的列向量
    B1 = B1.reshape((hidden_num, 1))
    B2 = B2.reshape((output_num, 1))

    # 计算隐藏层的激活值A1，使用双曲正切作为激活函数
    A1 = np.tanh(np.dot(W1, input_data) + np.tile(B1, (1, input_data.shape[1])))

    # 计算输出层的激活值A2，使用四舍五入作为激活函数
    A2 = np.round(np.dot(W2, A1) + np.tile(B2, (1, input_data.shape[1])))

    # 计算均方误差(MSE)作为适应度值
    mse_chrom = np.mean((A2.T - output_data) ** 2)

    # 最终的适应度值即为均方误差
    fitness_value = mse_chrom

    return fitness_value  # 返回适应度值


In [8]:
def cross(cross_pro, _a, fit_g_n):
    set_temp_new = []  # 创建一个空列表，用于存储新的个体
    for i in range(1, int(fit_g_n/2) + 1):  # 遍历适应度数量的一半次数
        p1 = _a[2 * i - 1, :]  # 获取父代1的基因
        p2 = _a[2 * i, :]      # 获取父代2的基因
        if np.random.rand() > cross_pro:  # 以交叉概率为阈值，判断是否进行交叉
            pick1 = np.random.randint(1, p1.shape[1] - 1)  # 随机选择交叉点1
            pick2 = np.random.randint(1, p1.shape[1] - 1)  # 随机选择交叉点2
            if pick1 > pick2:
                pick1, pick2 = pick2, pick1  # 确保pick1小于pick2
            child1 = np.hstack((p1[:, :pick1], p2[:, pick1:pick2], p1[:, pick2 + 1:]))  # 生成子代1
            child2 = np.hstack((p2[:, :pick1], p1[:, pick1:pick2], p2[:, pick2 + 1:]))  # 生成子代2
            set_temp_new.extend([child1, child2])  # 将子代添加到新个体列表中
    new_chrom = np.vstack((_a, set_temp_new))  # 将新个体与原个体合并成新种群
    return new_chrom  # 返回新种群


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义神经网络类
class Net(nn.Module):
    def __init__(self, input_num, hidden_num, output_num):
        super(Net, self).__init__()
        # 第一层全连接层：输入维度 -> 隐层维度
        self.fc1 = nn.Linear(input_num, hidden_num)
        # 激活函数：双曲正切函数
        self.tansig = nn.Tanh()
        # 第二层全连接层：隐层维度 -> 输出维度
        self.fc2 = nn.Linear(hidden_num, output_num)

    # 前向传播函数
    def forward(self, x):
        x = self.tansig(self.fc1(x))
        x = self.fc2(x)
        return x

# 定义BP神经网络训练和预测函数
def BP(x_train_regular, y_train_regular, hidden_num, input_num, output_num, chrom, x_test_data, x_train_maxmin, y_train_maxmin):
    # 创建神经网络实例
    net = Net(input_num, hidden_num, output_num)
    
    # 从染色体中提取权重和偏置项
    w1 = torch.Tensor(chrom[:input_num * hidden_num].reshape(hidden_num, input_num))
    B1 = torch.Tensor(chrom[input_num * hidden_num + 1:input_num * hidden_num + hidden_num].reshape(hidden_num, 1))
    w2 = torch.Tensor(chrom[input_num * hidden_num + hidden_num + 1:input_num * hidden_num + hidden_num + hidden_num * output_num].reshape(output_num, hidden_num))
    B2 = torch.Tensor(chrom[input_num * hidden_num + hidden_num + hidden_num * output_num + 1:].reshape(output_num, 1))
    
    # 将权重和偏置项设置到神经网络层中
    net.fc1.weight.data = w1
    net.fc1.bias.data = B1.squeeze()
    net.fc2.weight.data = w2
    net.fc2.bias.data = B2.squeeze()
    
    # 定义损失函数为均方误差
    criterion = nn.MSELoss()
    
    # 使用LBFGS优化器进行训练
    optimizer = optim.LBFGS(net.parameters(), lr=0.1)
    for epoch in range(10):
        def closure():
            optimizer.zero_grad()
            outputs = net(x_train_regular)
            loss = criterion(outputs, y_train_regular)
            loss.backward()
            return loss
        optimizer.step(closure)
    
    # 对测试数据进行归一化处理
    x_test_regular = (x_test_data - x_train_maxmin[0]) / (x_train_maxmin[1] - x_train_maxmin[0])
    
    # 使用训练好的神经网络进行预测
    y_test_regular = net(torch.Tensor(x_test_regular)).detach().numpy()
    
    # 将预测结果反归一化为原始范围
    GA_BP_predict = y_test_regular * (y_train_maxmin[1] - y_train_maxmin[0]) + y_train_maxmin[0]
    
    return GA_BP_predict


In [10]:
# 从Excel文件读取数据1，包含患者的影像信息、血肿及水肿的体积及位置
data1 = pd.read_excel('表2-患者影像信息血肿及水肿的体积及位置.xlsx')

# 从Excel文件读取数据2，包含患者的影像信息、血肿及水肿的形状及灰度分布，指定工作表为'Hemo'
data2 = pd.read_excel('表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx', sheet_name='Hemo')

# 使用左连接合并数据1和数据2，根据数据1的第185列与数据2的第1列进行连接，使用后缀'_'来区分重复列名
merged_data = data1.merge(data2, left_on=data1.iloc[:, 185], right_on=data2.iloc[:, 1], how='left', suffixes=('', '_y'))

# 选择需要的列，即数据2的所有列
columns_to_select = data2.columns

# 对合并后的数据中的每一列，在缺失值处填充0
for column in columns_to_select:
    merged_data[column] = merged_data[column].fillna(0)


In [11]:
import pandas as pd
import numpy as np

# 加载数据集
data = pd.read_excel('GABP数据集.xlsx')

# 定义参数
input_num = data.shape[1] - 1  # 输入特征数量
hidden_num = 6  # 隐藏层神经元数量
output_num = 1  # 输出特征数量
NIND = 50  # 种群中的个体数量
iterations = 50
crossover_prob = 0.7
mutation_prob = 0.3

# 初始化种群
init_popu = initialize(hidden_num, x_train_regular, y_train_regular, NIND)

# 定义变异的限制范围
limit = np.array([[-2, -2, ...], [2, 2, ...]])

for k in range(limit.shape[0]):
    limit[k, 0] += 0.1 * limit[k, 0]
    limit[k, 1] += 0.1 * limit[k, 1]
    if limit[k, 0] < -2:
        limit[k, 0] = -2
    if limit[k, 1] > 2:
        limit[k, 1] = 2

GAed_popu = np.copy(init_popu)

# 遗传算法
for d in range(iterations):
    # 交叉
    new_popu = cross(crossover_prob, GAed_popu, NIND)

    # 变异
    new_popu = mutation(mutation_prob, lenchrom, new_popu, NIND, limit)

    # 计算适应度
    new_fitness = []
    for j in range(NIND):
        sgroup = GAed_popu[j, :]
        new_fitness.append(fitness(sgroup, input_num, hidden_num, output_num, x_train_regular, y_train_regular))

    # 选择
    new_popu = select(new_popu, new_fitness, NIND)

# BP神经网络
MSE_GABP = []
MAE_GABP = []
MAPE_GABP = []
P = []

for nind in range(NIND):
    chrom = GAed_popu[nind, :]
    GA_BP_predict = BP(x_train_regular, y_train_regular, hidden_num, input_num, output_num, chrom,
                       x_test_data, x_train_maxmin, y_train_maxmin)
    P.append(GA_BP_predict)

# 后处理
P = np.array(P)
p1 = np.round(P[:, 0])

for i in range(p1.shape[0]):
    if p1[i, 0] < 0:
        p1[i, 0] = 0
    if p1[i, 0] > 6:
        p1[i, 0] = 6


In [12]:
data = pd.read_excel('前100名患者预测与对照值.xlsx')
color = [[0, 1, 1],
         [9, 147, 150],
         [145, 211, 192],
         [235, 215, 165],
         [238, 155, 0],
         [204, 102, 2],
         [188, 62, 3],
         [174, 32, 18],
         [155, 34, 39]]
fig, ax = plt.subplots()
ax.plot([1, 1, 100], data.iloc[:, 0], color=color[0], linewidth=1.5)
ax.plot([1, 1, 100], data.iloc[:, 1], color=color[5], linewidth=1.5)
ax.set_xlabel('患者', fontproperties='SimHei', fontsize=12)
ax.set_ylabel('mRS', fontproperties='SimHei', fontsize=15)
ax.set_xticklabels(ax.get_xticks(), fontproperties='SimHei', fontsize=12)
ax.set_yticklabels(ax.get_yticks(), fontproperties='SimHei', fontsize=15)
ax.legend(['预测值', '实际值'])
plt.show()